In [2]:
from rapidfuzz import process, fuzz

# Example label map (extend as needed)
label_map = {
    'CLAIMANT_NAME': ['Name of Claimant', 'Claimant Name', 'Name'],
    'FATHER_NAME': ['Name of Father', 'Name of Father/Spouse', 'Father Name'],
    'VILLAGE': ['Village', 'Village Name'],
    'DISTRICT': ['District'],
    'CLAIM_ID': ['Claim ID', 'Claim No', 'Claim Number']
}

# `lines` produced earlier
choices = [ln['text'] for ln in lines]

def fuzzy_find_label_line(patterns, choices, scorer=fuzz.partial_ratio, thresh=65):
    # Try every pattern and pick best
    best = None
    for p in patterns:
        res = process.extractOne(p, choices, scorer=scorer)
        if res and res[1] >= thresh:
            if not best or res[1] > best[1]:
                best = res
    return best  # (matched_text, score, idx)

# adjacency: find text to right on same line
def get_text_right(line):
    # naive: split by ':' and take right side
    if ':' in line['text']:
        return line['text'].split(':', 1)[1].strip()
    # else try to find words to the right by bbox
    return None

extracted = {}
for label_key, patterns in label_map.items():
    best = fuzzy_find_label_line(patterns, choices)
    if best:
        matched, score, _ = best
        idx = choices.index(matched)
        # get line dict
        line = lines[idx]
        val = get_text_right(line)
        if not val:
            # fallback: look for words to right via bboxes
            # find word in that line that matches label token start, then take subsequent words
            val = ''
        extracted[label_key] = val

print(extracted)

NameError: name 'lines' is not defined